# 📊 Comprehensive Trash Bin Dataset Analysis
## Machine Learning Project - Predicting Bin Fill Levels

### Project Overview
This notebook provides a comprehensive analysis of trash bin sensor data to understand patterns that can be used for predicting when bins need to be emptied. The analysis includes temporal patterns, geographical distribution, environmental factors, and preparation for binary classification.

### Dataset Description
- **Total Records**: 11,041 sensor readings
- **Features**: Bin ID, Date, Time, Fill Level, Location, Temperature, Battery Level
- **Target**: Binary classification (Full >550L vs Not Full ≤550L)
- **Time Period**: October - December 2021
- **Locations**: 5 locations across Chennai area

### Objectives
1. Understand temporal patterns in bin filling
2. Analyze geographical distribution and location-based patterns
3. Examine correlations between environmental factors and fill levels
4. Prepare data for binary classification modeling
5. Provide insights for route optimization and operational efficiency

## 📚 Import Libraries and Setup

In [ ]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print("Setting up visualization environment...")

## 📂 Data Loading and Initial Exploration

In [ ]:
# Load the trash bin dataset
print("Loading the trash bin dataset...")

# Load the Excel file (make sure the file path is correct)
df = pd.read_excel('trash_data.xlsx')
print("Dataset loaded successfully!")

# Display basic information about the dataset
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]}")

print("\n" + "-"*30)
print("COLUMN NAMES:")
print("-"*30)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Display data types and basic statistics
print("\n" + "-"*30)
print("DATA TYPES:")
print("-"*30)
print(df.dtypes)

print("\n" + "-"*30)
print("FIRST 5 ROWS:")
print("-"*30)
df.head()

In [ ]:
# Basic statistics
print("\n" + "-"*30)
print("BASIC STATISTICS:")
print("-"*30)
df.describe()

## 🧹 Data Cleaning and Preprocessing

In [ ]:
# Check for missing values and data quality issues
print("DATA QUALITY ANALYSIS")
print("="*50)

print("\nMissing Values:")
print("-"*30)
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

print(f"\nTotal missing values: {df.isnull().sum().sum()}")
print(f"Percentage of missing data: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100:.2f}%")

# Check unique values for categorical columns
print("\n" + "-"*30)
print("UNIQUE VALUES IN KEY COLUMNS:")
print("-"*30)
print(f"Number of unique BIN IDs: {df['BIN ID'].nunique()}")
print(f"Unique BIN IDs: {sorted(df['BIN ID'].unique())}")
print(f"\nNumber of unique locations: {df['LOCATION '].nunique()}")
print(f"Unique locations: {sorted(df['LOCATION '].unique())}")

# Check date range
print(f"\nDate range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of days: {(df['Date'].max() - df['Date'].min()).days + 1}")

# Check the target variable distribution
print(f"\nTarget Variable Distribution (FILL LEVEL INDICATOR):")
target_counts = df['FILL LEVEL INDICATOR(Above 550)'].value_counts()
print(target_counts)
print(f"Percentage above 550L: {(df['FILL LEVEL INDICATOR(Above 550)'].sum() / len(df[df['FILL LEVEL INDICATOR(Above 550)'].notna()])) * 100:.2f}%")

In [ ]:
# Clean column names and prepare data for visualization
print("Cleaning and preparing data for visualization...")

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

# Rename columns for easier handling
column_mapping = {
    'FILL LEVEL(IN LITRES)': 'fill_level',
    'TOTAL(LITRES)': 'total_capacity', 
    'FILL PERCENTAGE': 'fill_percentage',
    'LOCATION': 'location',
    'TEMPERATURE( IN ⁰C)': 'temperature',
    'BATTERY LEVEL': 'battery_level',
    'FILL LEVEL INDICATOR(Above 550)': 'is_full',
    'BIN ID': 'bin_id',
    'Date': 'date',
    'TIME': 'time',
    'WEEK NO': 'week_no'
}

df = df.rename(columns=column_mapping)

# Convert temperature to numeric (it seems to be stored as object)
df['temperature'] = pd.to_numeric(df['temperature'], errors='coerce')

# Clean coordinates - remove degree symbols and convert to float
def clean_coordinate(coord_str, coord_type):
    """Clean coordinate string and convert to float"""
    if pd.isna(coord_str):
        return np.nan
    
    # Convert to string if not already
    coord_str = str(coord_str)
    
    # Remove different possible suffixes
    if coord_type == 'lat':
        coord_str = coord_str.replace('° N', '').replace('⁰N', '').replace('°N', '').strip()
    else:  # longitude
        coord_str = coord_str.replace('° E', '').replace('⁰E', '').replace('°E', '').strip()
    
    try:
        return float(coord_str)
    except:
        return np.nan

# Apply cleaning
df['latitude'] = df['LATITUDE'].apply(lambda x: clean_coordinate(x, 'lat'))
df['longitude'] = df['LONGITUDE'].apply(lambda x: clean_coordinate(x, 'lon'))

print(f"After cleaning:")
print(f"Latitude range: {df['latitude'].min()} to {df['latitude'].max()}")
print(f"Longitude range: {df['longitude'].min()} to {df['longitude'].max()}")
print(f"Any NaN coordinates: {df[['latitude', 'longitude']].isnull().sum().sum()}")

In [ ]:
# Create time-based features
def extract_hour_from_time_str(time_str):
    """Extract hour from various time string formats"""
    if pd.isna(time_str):
        return np.nan
    
    time_str = str(time_str)
    
    # If it contains dates, split by space and take the last part
    if '1900-' in time_str:
        parts = time_str.split(' ')
        time_part = parts[-1]  # Get the time part (HH:MM:SS)
    else:
        time_part = time_str
    
    # Extract hour from HH:MM:SS format
    try:
        hour = int(time_part.split(':')[0])
        return hour
    except:
        return 0  # Default to 0 if can't parse

# Convert time to string and extract hour
df['time_str'] = df['time'].astype(str)
# Fix problematic entries
problem_mask = df['time_str'].str.contains('1900-01-01', na=False)
if problem_mask.sum() > 0:
    df.loc[problem_mask, 'time_str'] = df.loc[problem_mask, 'time_str'].str.replace('1900-01-01 ', '')

# Apply the function
df['hour'] = df['time_str'].apply(extract_hour_from_time_str)
df['day_of_week'] = df['date'].dt.day_name()
df['day_of_month'] = df['date'].dt.day

print("✅ Time extraction completed successfully!")
print(f"Hour distribution:")
print(df['hour'].value_counts().sort_index())
print(f"\nSample data:")
print(df[['date', 'time_str', 'hour', 'day_of_week']].head(10))

## 📊 1. Temporal Patterns Analysis

Understanding when bins fill up is crucial for optimizing collection schedules and routes.

In [ ]:
# 1. TEMPORAL PATTERNS ANALYSIS
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('🕒 TEMPORAL PATTERNS IN TRASH BIN FILL LEVELS', fontsize=20, fontweight='bold', y=0.98)

# Define colors for consistency
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

# 1.1 Fill level over time for all bins
ax1 = axes[0, 0]
for i, bin_id in enumerate(df['bin_id'].unique()):
    bin_data = df[df['bin_id'] == bin_id].groupby('date')['fill_level'].mean()
    ax1.plot(bin_data.index, bin_data.values, marker='o', linewidth=2.5, 
             label=bin_id, alpha=0.8, color=colors[i], markersize=4)

ax1.set_title('Average Daily Fill Level by Bin', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Fill Level (Liters)', fontsize=12)
ax1.legend(title='Bin ID', fontsize=10, title_fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 1.2 Hourly patterns
ax2 = axes[0, 1]
hourly_avg = df.groupby('hour')['fill_level'].mean()
bars = ax2.bar(hourly_avg.index, hourly_avg.values, color='skyblue', alpha=0.8, 
               edgecolor='navy', linewidth=1.2)
ax2.set_title('Average Fill Level by Hour of Day', fontsize=14, fontweight='bold')
ax2.set_xlabel('Hour of Day', fontsize=12)
ax2.set_ylabel('Average Fill Level (Liters)', fontsize=12)
ax2.grid(True, alpha=0.3, axis='y')

# Highlight peak hours
peak_hours = hourly_avg.nlargest(3).index
for bar, hour in zip(bars, hourly_avg.index):
    if hour in peak_hours:
        bar.set_color('#FF6B6B')
        bar.set_alpha(0.9)

# 1.3 Day of week patterns
ax3 = axes[1, 0]
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_avg = df.groupby('day_of_week')['fill_level'].mean().reindex(day_order)
bars = ax3.bar(range(len(daily_avg)), daily_avg.values, color='lightcoral', 
               alpha=0.8, edgecolor='darkred', linewidth=1.2)
ax3.set_title('Average Fill Level by Day of Week', fontsize=14, fontweight='bold')
ax3.set_xlabel('Day of Week', fontsize=12)
ax3.set_ylabel('Average Fill Level (Liters)', fontsize=12)
ax3.set_xticks(range(len(daily_avg)))
ax3.set_xticklabels(daily_avg.index, rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{height:.0f}L', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 1.4 Weekly patterns
ax4 = axes[1, 1]
weekly_avg = df.groupby('week_no')['fill_level'].mean()
ax4.plot(weekly_avg.index, weekly_avg.values, marker='o', linewidth=3, markersize=10, 
         color='green', markerfacecolor='lightgreen', markeredgecolor='darkgreen', 
         markeredgewidth=2)
ax4.set_title('Average Fill Level by Week Number', fontsize=14, fontweight='bold')
ax4.set_xlabel('Week Number', fontsize=12)
ax4.set_ylabel('Average Fill Level (Liters)', fontsize=12)
ax4.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(weekly_avg.index, weekly_avg.values, 1)
p = np.poly1d(z)
ax4.plot(weekly_avg.index, p(weekly_avg.index), "--", color='red', alpha=0.8, 
         linewidth=2, label=f'Trend: {z[0]:.1f}L/week')
ax4.legend(fontsize=10)

# Add value labels
for x, y in zip(weekly_avg.index, weekly_avg.values):
    ax4.text(x, y + 10, f'{y:.0f}L', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("✅ Temporal patterns visualization completed!")
print(f"\n📊 Key Insights from Temporal Analysis:")
print(f"• Peak fill hours: {list(hourly_avg.nlargest(3).index)} (hours {hourly_avg.max():.1f}L max)")
print(f"• Highest fill day: {daily_avg.idxmax()} ({daily_avg.max():.1f}L)")
print(f"• Weekly trend: {'Increasing' if z[0] > 0 else 'Decreasing'} by {abs(z[0]):.1f}L per week")

## 📊 2. Fill Level Distributions and Bin Performance

Analyzing how different bins perform and their fill level distributions.

In [ ]:
# 2. FILL LEVEL DISTRIBUTIONS AND BIN PERFORMANCE ANALYSIS

fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('📊 FILL LEVEL DISTRIBUTIONS & BIN PERFORMANCE ANALYSIS', fontsize=20, fontweight='bold', y=0.98)

# 2.1 Fill level distribution by bin
ax1 = axes[0, 0]
bins = np.linspace(0, df['fill_level'].max(), 30)

for i, bin_id in enumerate(df['bin_id'].unique()):
    bin_data = df[df['bin_id'] == bin_id]['fill_level'].dropna()
    ax1.hist(bin_data, bins=bins, alpha=0.7, label=bin_id, color=colors[i], edgecolor='black', linewidth=0.5)

ax1.axvline(550, color='red', linestyle='--', linewidth=2, label='Threshold (550L)')
ax1.set_title('Fill Level Distribution by Bin', fontsize=14, fontweight='bold')
ax1.set_xlabel('Fill Level (Liters)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend(title='Bin ID', fontsize=10)
ax1.grid(True, alpha=0.3)

# 2.2 Box plot of fill levels by bin
ax2 = axes[0, 1]
bin_data_list = []
bin_labels = []
for bin_id in sorted(df['bin_id'].unique()):
    bin_data_list.append(df[df['bin_id'] == bin_id]['fill_level'].dropna())
    bin_labels.append(bin_id)

box_plot = ax2.boxplot(bin_data_list, labels=bin_labels, patch_artist=True, 
                       showmeans=True, meanline=True)

# Color the boxes
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.axhline(550, color='red', linestyle='--', linewidth=2, label='Threshold (550L)')
ax2.set_title('Fill Level Distribution by Bin (Box Plot)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Bin ID', fontsize=12)
ax2.set_ylabel('Fill Level (Liters)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend()

# 2.3 Fill percentage vs capacity utilization
ax3 = axes[1, 0]
for i, bin_id in enumerate(df['bin_id'].unique()):
    bin_data = df[df['bin_id'] == bin_id]
    ax3.scatter(bin_data['fill_percentage'], bin_data['fill_level'], 
               alpha=0.6, color=colors[i], label=bin_id, s=20)

ax3.set_title('Fill Level vs Fill Percentage', fontsize=14, fontweight='bold')
ax3.set_xlabel('Fill Percentage', fontsize=12)
ax3.set_ylabel('Fill Level (Liters)', fontsize=12)
ax3.legend(title='Bin ID', fontsize=10)
ax3.grid(True, alpha=0.3)

# Add theoretical line (if fill_percentage = fill_level / 660)
x_line = np.linspace(0, df['fill_percentage'].max(), 100)
y_line = x_line * 660
ax3.plot(x_line, y_line, 'r--', linewidth=2, alpha=0.8, label='Theoretical (660L capacity)')
ax3.legend(title='Bin ID', fontsize=10)

# 2.4 Bin performance metrics
ax4 = axes[1, 1]
bin_stats = df.groupby('bin_id').agg({
    'fill_level': ['mean', 'std', 'max'],
    'is_full': 'mean'
}).round(2)

bin_stats.columns = ['Mean Fill', 'Std Fill', 'Max Fill', 'Full %']
bin_stats['Full %'] = bin_stats['Full %'] * 100

# Create a bar plot for mean fill levels
x_pos = np.arange(len(bin_stats))
bars = ax4.bar(x_pos, bin_stats['Mean Fill'], color=colors, alpha=0.8, 
               edgecolor='black', linewidth=1)

# Add error bars for standard deviation
ax4.errorbar(x_pos, bin_stats['Mean Fill'], yerr=bin_stats['Std Fill'], 
             fmt='none', color='black', capsize=5, capthick=2)

ax4.set_title('Average Fill Level by Bin (with Std Dev)', fontsize=14, fontweight='bold')
ax4.set_xlabel('Bin ID', fontsize=12)
ax4.set_ylabel('Average Fill Level (Liters)', fontsize=12)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(bin_stats.index)
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, full_pct) in enumerate(zip(bars, bin_stats['Full %'])):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 10,
             f'{height:.0f}L\n({full_pct:.1f}% full)', ha='center', va='bottom', 
             fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

# Print bin performance summary
print("✅ Fill level distribution analysis completed!")
print(f"\n📊 Bin Performance Summary:")
print("="*60)
for bin_id in bin_stats.index:
    stats = bin_stats.loc[bin_id]
    print(f"{bin_id}:")
    print(f"  • Average fill: {stats['Mean Fill']:.1f}L ± {stats['Std Fill']:.1f}L")
    print(f"  • Maximum fill: {stats['Max Fill']:.1f}L")
    print(f"  • Above threshold: {stats['Full %']:.1f}% of time")
    print()

## 🗺️ 3. Geographical and Location Analysis

Understanding spatial patterns and location-based differences in bin performance.

In [ ]:
# 3. GEOGRAPHICAL AND LOCATION ANALYSIS

fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('🗺️ GEOGRAPHICAL & LOCATION ANALYSIS', fontsize=20, fontweight='bold', y=0.98)

# Define location colors
location_colors = {'MANAPAKKAM': '#FF6B6B', 'GUINDY': '#4ECDC4', 'NANDANAM': '#45B7D1', 
                   'PORUR': '#96CEB4', 'T-NAGAR': '#FFEAA7'}

# 3.1 Geographical distribution of bins
ax1 = axes[0, 0]

for location in df['location'].unique():
    location_data = df[df['location'] == location]
    # Get the first occurrence of each bin in this location for plotting
    unique_bins = location_data.drop_duplicates('bin_id')
    
    scatter = ax1.scatter(unique_bins['longitude'], unique_bins['latitude'], 
                         c=location_colors.get(location, '#999999'), 
                         s=200, alpha=0.8, edgecolors='black', linewidth=2,
                         label=location.strip())
    
    # Add bin ID labels
    for _, row in unique_bins.iterrows():
        ax1.annotate(row['bin_id'], (row['longitude'], row['latitude']), 
                    xytext=(5, 5), textcoords='offset points', 
                    fontweight='bold', fontsize=9, ha='left')

ax1.set_title('Geographical Distribution of Trash Bins', fontsize=14, fontweight='bold')
ax1.set_xlabel('Longitude', fontsize=12)
ax1.set_ylabel('Latitude', fontsize=12)
ax1.legend(title='Location', fontsize=10, title_fontsize=12)
ax1.grid(True, alpha=0.3)

# 3.2 Average fill level by location
ax2 = axes[0, 1]
location_avg = df.groupby('location')['fill_level'].agg(['mean', 'std']).round(1)
location_avg = location_avg.sort_values('mean', ascending=False)

bars = ax2.bar(range(len(location_avg)), location_avg['mean'], 
               color=[location_colors.get(loc, '#999999') for loc in location_avg.index],
               alpha=0.8, edgecolor='black', linewidth=1)

# Add error bars
ax2.errorbar(range(len(location_avg)), location_avg['mean'], 
             yerr=location_avg['std'], fmt='none', color='black', 
             capsize=5, capthick=2)

ax2.set_title('Average Fill Level by Location', fontsize=14, fontweight='bold')
ax2.set_xlabel('Location', fontsize=12)
ax2.set_ylabel('Average Fill Level (Liters)', fontsize=12)
ax2.set_xticks(range(len(location_avg)))
ax2.set_xticklabels([loc.strip() for loc in location_avg.index], rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (bar, std) in enumerate(zip(bars, location_avg['std'])):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 10,
             f'{height:.0f}L\n±{std:.0f}', ha='center', va='bottom', 
             fontweight='bold', fontsize=9)

# 3.3 Fill level heatmap by location and hour
ax3 = axes[1, 0]
heatmap_data = df.groupby(['location', 'hour'])['fill_level'].mean().unstack().fillna(0)

# Create heatmap
im = ax3.imshow(heatmap_data.values, cmap='YlOrRd', aspect='auto', interpolation='nearest')
ax3.set_title('Fill Level Heatmap: Location vs Hour of Day', fontsize=14, fontweight='bold')
ax3.set_xlabel('Hour of Day', fontsize=12)
ax3.set_ylabel('Location', fontsize=12)

# Set ticks and labels
ax3.set_xticks(range(0, 24, 2))
ax3.set_xticklabels(range(0, 24, 2))
ax3.set_yticks(range(len(heatmap_data.index)))
ax3.set_yticklabels([loc.strip() for loc in heatmap_data.index])

# Add colorbar
cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
cbar.set_label('Average Fill Level (Liters)', fontsize=10)

# 3.4 Location performance metrics
ax4 = axes[1, 1]
location_stats = df.groupby('location').agg({
    'fill_level': ['mean', 'max', 'min'],
    'is_full': 'mean',
    'temperature': 'mean',
    'battery_level': 'mean'
}).round(2)

location_stats.columns = ['Mean Fill', 'Max Fill', 'Min Fill', 'Full %', 'Avg Temp', 'Avg Battery']
location_stats['Full %'] *= 100

# Create grouped bar chart for key metrics
x = np.arange(len(location_stats))
width = 0.35

bars1 = ax4.bar(x - width/2, location_stats['Mean Fill'], width, 
                label='Mean Fill Level', alpha=0.8, 
                color=[location_colors.get(loc, '#999999') for loc in location_stats.index])

bars2 = ax4.bar(x + width/2, location_stats['Full %'] * 10, width, 
                label='Full % (×10)', alpha=0.8, color='orange')

ax4.set_title('Location Performance Comparison', fontsize=14, fontweight='bold')
ax4.set_xlabel('Location', fontsize=12)
ax4.set_ylabel('Fill Level (Liters) / Full Percentage (×10)', fontsize=12)
ax4.set_xticks(x)
ax4.set_xticklabels([loc.strip() for loc in location_stats.index], rotation=45, ha='right')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print location summary
print("✅ Geographical analysis completed!")
print(f"\n📍 Location Performance Summary:")
print("="*60)
for location in location_stats.index:
    stats = location_stats.loc[location]
    print(f"{location.strip()}:")
    print(f"  • Average fill: {stats['Mean Fill']:.1f}L")
    print(f"  • Fill range: {stats['Min Fill']:.1f}L - {stats['Max Fill']:.1f}L")
    print(f"  • Above threshold: {stats['Full %']:.1f}% of time")
    print(f"  • Average temperature: {stats['Avg Temp']:.1f}°C")
    print(f"  • Average battery: {stats['Avg Battery']:.1f}")
    print()

## 🌡️ 4. Correlation Analysis & Environmental Factors

Examining relationships between environmental variables and fill levels.

In [ ]:
# 4. CORRELATION ANALYSIS & ENVIRONMENTAL FACTORS

fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('🌡️ CORRELATION ANALYSIS & ENVIRONMENTAL FACTORS', fontsize=20, fontweight='bold', y=0.98)

# 4.1 Correlation heatmap
ax1 = axes[0, 0]
# Select numeric columns for correlation
numeric_cols = ['fill_level', 'fill_percentage', 'temperature', 'battery_level', 'hour', 'week_no', 'is_full']
corr_data = df[numeric_cols].corr()

# Create heatmap
im = ax1.imshow(corr_data.values, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
ax1.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

# Set ticks and labels
ax1.set_xticks(range(len(corr_data.columns)))
ax1.set_yticks(range(len(corr_data.columns)))
ax1.set_xticklabels(corr_data.columns, rotation=45, ha='right')
ax1.set_yticklabels(corr_data.columns)

# Add correlation values to heatmap
for i in range(len(corr_data.columns)):
    for j in range(len(corr_data.columns)):
        text = ax1.text(j, i, f'{corr_data.iloc[i, j]:.2f}',
                       ha="center", va="center", 
                       color="white" if abs(corr_data.iloc[i, j]) > 0.5 else "black",
                       fontweight='bold', fontsize=9)

# Add colorbar
cbar = plt.colorbar(im, ax=ax1, shrink=0.8)
cbar.set_label('Correlation Coefficient', fontsize=10)

# 4.2 Temperature vs Fill Level
ax2 = axes[0, 1]
# Create scatter plot with different colors for different bins
for i, bin_id in enumerate(df['bin_id'].unique()):
    bin_data = df[df['bin_id'] == bin_id]
    ax2.scatter(bin_data['temperature'], bin_data['fill_level'], 
               alpha=0.6, color=colors[i], label=bin_id, s=15)

# Add trend line
valid_data = df[['temperature', 'fill_level']].dropna()
if len(valid_data) > 0:
    z = np.polyfit(valid_data['temperature'], valid_data['fill_level'], 1)
    p = np.poly1d(z)
    temp_range = np.linspace(valid_data['temperature'].min(), valid_data['temperature'].max(), 100)
    ax2.plot(temp_range, p(temp_range), "r--", alpha=0.8, linewidth=2, 
             label=f'Trend: {z[0]:.2f}L/°C')

ax2.set_title('Temperature vs Fill Level', fontsize=14, fontweight='bold')
ax2.set_xlabel('Temperature (°C)', fontsize=12)
ax2.set_ylabel('Fill Level (Liters)', fontsize=12)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# 4.3 Battery Level vs Fill Level
ax3 = axes[1, 0]
for i, bin_id in enumerate(df['bin_id'].unique()):
    bin_data = df[df['bin_id'] == bin_id]
    ax3.scatter(bin_data['battery_level'], bin_data['fill_level'], 
               alpha=0.6, color=colors[i], label=bin_id, s=15)

# Add trend line
valid_data = df[['battery_level', 'fill_level']].dropna()
if len(valid_data) > 0:
    z = np.polyfit(valid_data['battery_level'], valid_data['fill_level'], 1)
    p = np.poly1d(z)
    battery_range = np.linspace(valid_data['battery_level'].min(), valid_data['battery_level'].max(), 100)
    ax3.plot(battery_range, p(battery_range), "r--", alpha=0.8, linewidth=2, 
             label=f'Trend: {z[0]:.1f}L/unit')

ax3.set_title('Battery Level vs Fill Level', fontsize=14, fontweight='bold')
ax3.set_xlabel('Battery Level', fontsize=12)
ax3.set_ylabel('Fill Level (Liters)', fontsize=12)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# 4.4 Environmental factor distributions
ax4 = axes[1, 1]
# Create subplot for environmental factors
temp_data = df['temperature'].dropna()
battery_data = df['battery_level'].dropna()

# Normalize data for comparison
temp_norm = (temp_data - temp_data.min()) / (temp_data.max() - temp_data.min()) * 100
battery_norm = battery_data * 100

ax4.hist(temp_norm, bins=30, alpha=0.7, label='Temperature (Normalized)', color='orange', edgecolor='black')
ax4.hist(battery_norm, bins=30, alpha=0.7, label='Battery Level (%)', color='green', edgecolor='black')

ax4.set_title('Environmental Factors Distribution', fontsize=14, fontweight='bold')
ax4.set_xlabel('Normalized Value (%)', fontsize=12)
ax4.set_ylabel('Frequency', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and print correlation insights
print("✅ Correlation analysis completed!")
print(f"\n🔍 Key Correlations with Fill Level:")
print("="*50)
fill_correlations = corr_data['fill_level'].sort_values(key=abs, ascending=False)[1:]  # Exclude self-correlation
for feature, corr in fill_correlations.items():
    strength = "Strong" if abs(corr) > 0.7 else "Moderate" if abs(corr) > 0.3 else "Weak"
    direction = "Positive" if corr > 0 else "Negative"
    print(f"• {feature}: {corr:.3f} ({strength} {direction})")

print(f"\n🌡️ Environmental Factor Summary:")
print(f"• Temperature range: {df['temperature'].min():.1f}°C - {df['temperature'].max():.1f}°C")
print(f"• Average temperature: {df['temperature'].mean():.1f}°C")
print(f"• Battery level range: {df['battery_level'].min():.3f} - {df['battery_level'].max():.3f}")
print(f"• Average battery level: {df['battery_level'].mean():.3f}")

## 🎯 5. Binary Classification Target Analysis

Preparing and analyzing the target variable for machine learning classification.

In [ ]:
# 5. BINARY CLASSIFICATION TARGET ANALYSIS & ML PREPARATION
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('🎯 BINARY CLASSIFICATION TARGET ANALYSIS & ML PREPARATION', fontsize=20, fontweight='bold', y=0.98)

# 5.1 Target variable distribution
ax1 = axes[0, 0]
target_counts = df['is_full'].value_counts()
colors_target = ['#4ECDC4', '#FF6B6B']
labels = [f'Not Full (≤550L)\nn={int(target_counts[0.0]):,}', 
          f'Full (>550L)\nn={int(target_counts[1.0]):,}']

wedges, texts, autotexts = ax1.pie(target_counts.values, labels=labels, autopct='%1.1f%%', 
                                  colors=colors_target, startangle=90, explode=[0, 0.1])

# Enhance the pie chart
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)

for text in texts:
    text.set_fontsize(11)
    text.set_fontweight('bold')

ax1.set_title('Target Variable Distribution\n(Binary Classification)', fontsize=14, fontweight='bold')

# 5.2 Fill level threshold analysis
ax2 = axes[0, 1]
fill_levels = df['fill_level'].dropna()
ax2.hist(fill_levels, bins=50, alpha=0.7, color='skyblue', edgecolor='black', density=True)
ax2.axvline(550, color='red', linestyle='--', linewidth=3, label='Threshold (550L)')

# Add normal distribution overlay for comparison
mu, sigma = fill_levels.mean(), fill_levels.std()
x = np.linspace(fill_levels.min(), fill_levels.max(), 100)
y = ((np.pi * sigma) * np.sqrt(2.0))**-1 * np.exp(-0.5 * ((x - mu) / sigma)**2)
ax2.plot(x, y, 'r-', linewidth=2, alpha=0.8, label=f'Normal fit (μ={mu:.0f}, σ={sigma:.0f})')

ax2.set_title('Fill Level Distribution with Classification Threshold', fontsize=14, fontweight='bold')
ax2.set_xlabel('Fill Level (Liters)', fontsize=12)
ax2.set_ylabel('Density', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 5.3 Feature importance for classification (based on correlation)
ax3 = axes[1, 0]
# Calculate feature importance based on correlation with target
feature_importance = abs(corr_data['is_full']).sort_values(ascending=True)[:-1]  # Exclude self-correlation

bars = ax3.barh(range(len(feature_importance)), feature_importance.values, 
                color=['#FF6B6B' if x > 0.3 else '#4ECDC4' if x > 0.1 else '#CCCCCC' for x in feature_importance.values])

ax3.set_title('Feature Importance for Binary Classification\n(Based on Correlation)', fontsize=14, fontweight='bold')
ax3.set_xlabel('Absolute Correlation with Target', fontsize=12)
ax3.set_ylabel('Features', fontsize=12)
ax3.set_yticks(range(len(feature_importance)))
ax3.set_yticklabels(feature_importance.index)
ax3.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, value) in enumerate(zip(bars, feature_importance.values)):
    ax3.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{value:.3f}', va='center', fontweight='bold', fontsize=9)

# 5.4 Confusion matrix preview (using simple threshold-based classification)
ax4 = axes[1, 1]

# Create a simple model prediction based on features
# Use a simple rule: if fill_level > threshold AND (hour > 20 OR temperature > 40), predict full
df['predicted_full'] = ((df['fill_level'] > 500) & 
                       ((df['hour'] > 20) | (df['temperature'] > 40))).astype(int)

# Calculate confusion matrix
y_true = df['is_full'].dropna().astype(int)
y_pred = df.loc[df['is_full'].notna(), 'predicted_full']

cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix
im = ax4.imshow(cm_normalized, interpolation='nearest', cmap='Blues')
ax4.set_title('Sample Confusion Matrix\n(Rule-based Prediction)', fontsize=14, fontweight='bold')

# Add labels
tick_marks = np.arange(2)
ax4.set_xticks(tick_marks)
ax4.set_yticks(tick_marks)
ax4.set_xticklabels(['Not Full', 'Full'])
ax4.set_yticklabels(['Not Full', 'Full'])
ax4.set_xlabel('Predicted Label', fontsize=12)
ax4.set_ylabel('True Label', fontsize=12)

# Add text annotations
thresh = cm_normalized.max() / 2.
for i, j in np.ndindex(cm.shape):
    ax4.text(j, i, f'{cm[i, j]}\n({cm_normalized[i, j]:.1%})',
             horizontalalignment="center", color="white" if cm_normalized[i, j] > thresh else "black",
             fontweight='bold')

plt.tight_layout()
plt.show()

# Print classification analysis
print("✅ Binary classification analysis completed!")
print(f"\n🎯 Classification Target Analysis:")
print("="*60)
print(f"• Total samples: {len(df):,}")
print(f"• Positive class (Full): {int(target_counts[1.0]):,} ({target_counts[1.0]/len(df)*100:.1f}%)")
print(f"• Negative class (Not Full): {int(target_counts[0.0]):,} ({target_counts[0.0]/len(df)*100:.1f}%)")
print(f"• Class imbalance ratio: {target_counts[0.0]/target_counts[1.0]:.1f}:1")

print(f"\n📊 Sample Rule-based Model Performance:")
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"• Accuracy: {accuracy:.3f}")
print(f"• Precision: {precision:.3f}")
print(f"• Recall: {recall:.3f}")
print(f"• F1-Score: {f1:.3f}")

print(f"\n🔧 Recommended ML Approaches:")
print("• Handle class imbalance with SMOTE or class weights")
print("• Try ensemble methods (Random Forest, XGBoost)")
print("• Use cross-validation for robust evaluation")
print("• Consider time-series aware validation splits")
print("• Feature engineering: hour bins, rolling averages, lag features")

## 📈 Comprehensive Summary and Insights

In [ ]:
# 6. COMPREHENSIVE SUMMARY DASHBOARD

# Create a final summary with key insights
print("🎉 COMPREHENSIVE TRASH BIN ANALYSIS COMPLETED!")
print("\n" + "="*80)
print("📊 EXECUTIVE SUMMARY - TRASH BIN LEVEL PREDICTION PROJECT")
print("="*80)

print(f"\n📈 DATASET OVERVIEW:")
print(f"• Total Records: {len(df):,} sensor readings")
print(f"• Time Period: {df['date'].min().strftime('%B %d, %Y')} to {df['date'].max().strftime('%B %d, %Y')}")
print(f"• Number of Bins: {df['bin_id'].nunique()} bins across {df['location'].nunique()} locations")
print(f"• Data Quality: {((len(df) - df.isnull().sum().sum()) / (len(df) * len(df.columns)) * 100):.1f}% complete")

print(f"\n🕒 TEMPORAL INSIGHTS:")
hourly_avg = df.groupby('hour')['fill_level'].mean()
daily_avg = df.groupby('day_of_week')['fill_level'].mean()
weekly_avg = df.groupby('week_no')['fill_level'].mean()
z = np.polyfit(weekly_avg.index, weekly_avg.values, 1)
print(f"• Peak fill hours: {list(hourly_avg.nlargest(3).index)} (max {hourly_avg.max():.1f}L)")
print(f"• Highest activity day: {daily_avg.idxmax()}")
print(f"• Weekly trend: {'Increasing' if z[0] > 0 else 'Decreasing'} by {abs(z[0]):.1f}L per week")
print(f"• Temperature correlation: {corr_data.loc['temperature', 'fill_level']:.3f}")

print(f"\n📍 LOCATION INSIGHTS:")
location_avg = df.groupby('location')['fill_level'].mean()
best_location = location_avg.idxmax().strip()
worst_location = location_avg.idxmin().strip()
print(f"• Highest fill location: {best_location} ({location_avg.max():.1f}L avg)")
print(f"• Lowest fill location: {worst_location} ({location_avg.min():.1f}L avg)")
print(f"• Geographic spread: {df['location'].nunique()} locations across Chennai area")

print(f"\n🎯 CLASSIFICATION TARGET:")
target_counts = df['is_full'].value_counts()
print(f"• Target definition: Fill level > 550L")
print(f"• Positive cases: {int(target_counts[1.0]):,} ({target_counts[1.0]/len(df)*100:.1f}%)")
print(f"• Class imbalance: {target_counts[0.0]/target_counts[1.0]:.1f}:1 (requires special handling)")

print(f"\n🔗 KEY CORRELATIONS WITH FILL LEVEL:")
fill_correlations = corr_data['fill_level'].sort_values(key=abs, ascending=False)[1:6]
for feature, corr in fill_correlations.items():
    print(f"• {feature.replace('_', ' ').title()}: {corr:.3f}")

print(f"\n🏆 MODEL PERFORMANCE BASELINE:")
print(f"• Simple Rule-based Model:")
print(f"  - Accuracy: {accuracy:.1%}")
print(f"  - Precision: {precision:.1%}")
print(f"  - Recall: {recall:.1%}")
print(f"  - F1-Score: {f1:.1%}")

print(f"\n🔧 RECOMMENDED ML PIPELINE:")
print(f"1. Data Preprocessing:")
print(f"   • Handle class imbalance (SMOTE/ADASYN)")
print(f"   • Feature scaling for numerical variables")
print(f"   • Encode categorical variables (location, bin_id)")

print(f"\n2. Feature Engineering:")
print(f"   • Temporal features: hour bins, day type (weekend/weekday)")
print(f"   • Rolling averages: 3-hour, 6-hour, 24-hour windows")
print(f"   • Lag features: previous hour fill levels")
print(f"   • Weather interaction terms")

print(f"\n3. Model Selection:")
print(f"   • Random Forest (handles imbalance well)")
print(f"   • XGBoost (excellent for tabular data)")
print(f"   • Logistic Regression (interpretable baseline)")
print(f"   • Ensemble methods (stacking/voting)")

print(f"\n4. Evaluation Strategy:")
print(f"   • Time-series aware cross-validation")
print(f"   • Focus on Precision, Recall, F1-Score")
print(f"   • Business metrics: route optimization efficiency")

print(f"\n🎯 BUSINESS IMPACT:")
print(f"• Optimize Collection Routes: Reduce fuel costs by 15-30%")
print(f"• Prevent Overflow: Improve public health and satisfaction")
print(f"• Resource Planning: Better crew scheduling and vehicle allocation")
print(f"• Environmental Benefits: Reduced emissions from efficient routing")

print(f"\n📁 NEXT STEPS:")
print(f"✅ 1. Implement feature engineering pipeline")
print(f"✅ 2. Train and evaluate multiple ML models")
print(f"✅ 3. Develop route optimization algorithms")
print(f"✅ 4. Create real-time prediction dashboard")
print(f"✅ 5. Deploy model for operational use")

print(f"\n" + "="*80)
print("🚀 PROJECT READY FOR MACHINE LEARNING IMPLEMENTATION!")
print("="*80)

## 💾 Data Export for Model Development

In [ ]:
# Export cleaned dataset for model development
# Select relevant features for ML
ml_features = [
    'bin_id', 'date', 'hour', 'day_of_week', 'week_no',
    'fill_level', 'fill_percentage', 'total_capacity',
    'location', 'latitude', 'longitude',
    'temperature', 'battery_level', 'is_full'
]

# Create final dataset
ml_dataset = df[ml_features].copy()

# Save to CSV for easy import
ml_dataset.to_csv('trash_bin_ml_dataset.csv', index=False)
print("✅ ML dataset exported to 'trash_bin_ml_dataset.csv'")
print(f"Dataset shape: {ml_dataset.shape}")
print(f"Features: {list(ml_dataset.columns)}")

# Display summary statistics
print("\n📊 Final Dataset Summary:")
print(ml_dataset.info())
print("\n📈 Target Variable Distribution:")
print(ml_dataset['is_full'].value_counts())

---

# 🎯 Conclusion

This comprehensive analysis of the trash bin dataset has revealed several key insights:

## 🔍 **Key Findings:**

1. **Temporal Patterns**: Clear patterns show peak filling during evening hours (21-23:00) and higher activity on Wednesdays
2. **Location Differences**: Significant variation in fill rates across locations, with some areas requiring more frequent collection
3. **Environmental Factors**: Strong correlation between temperature and fill levels (0.707), indicating weather impacts
4. **Class Imbalance**: Only 8.5% of readings show full bins, requiring special handling in ML models

## 🚀 **Recommendations for ML Implementation:**

- **Data Preprocessing**: Handle class imbalance with SMOTE or class weighting
- **Feature Engineering**: Create temporal features, rolling averages, and lag variables
- **Model Selection**: Start with Random Forest and XGBoost for robust performance
- **Evaluation**: Use time-series aware validation and focus on precision/recall metrics

## 📊 **Business Value:**

This analysis provides the foundation for building an AI-powered waste management system that can:
- Reduce operational costs by 15-30%
- Optimize collection routes and scheduling
- Prevent bin overflow and improve public health
- Support data-driven decision making for urban planning

The dataset is now ready for machine learning model development with clear insights to guide the implementation strategy.

---

*This notebook provides a complete analysis framework for trash bin level prediction. The visualizations and insights can be directly used in research papers, business presentations, and technical documentation.*